In [1]:
import cv2
from ultralytics import YOLO
import numpy as np # Cần cho numpy array operations

# --- Cấu hình ---
YOLO_MODEL_PATH = "yolov8n.pt"
TARGET_CLASS_ID = 0  # Class ID cho 'person' trong COCO
CONFIDENCE_THRESHOLD = 0.45
K_DISTANCE_FACTOR = 70000 # Bạn cần hiệu chỉnh K_DISTANCE_FACTOR này!
                         # Nếu D_known_meters = 2m, Area_observed_pixels = 35000 px²
                         # K = 2 * 35000 = 70000 (để distance ra mét)

# --- Khởi tạo ---
model = YOLO(YOLO_MODEL_PATH)
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Lỗi: Không thể mở webcam.")
    exit()

print("Webcam đã được mở. Nhấn 'ESC' để thoát.")

# Thông số cho các text chi tiết (width, height, area)
FONT_DETAILS = cv2.FONT_HERSHEY_SIMPLEX
FONT_SCALE_DETAILS = 0.45
COLOR_DETAILS = (50, 205, 50)  # Màu LimeGreen
THICKNESS_DETAILS = 1
LINE_SPACING_DETAILS = 18

while True:
    ret, frame_bgr = cap.read()
    if not ret:
        print("Lỗi: Không thể đọc frame từ webcam. Thoát.")
        break

    results = model(frame_bgr, classes=[TARGET_CLASS_ID], conf=CONFIDENCE_THRESHOLD, verbose=False)

    # Biến để lưu thông tin của người có diện tích lớn nhất (gần nhất)
    largest_area_person_info = None
    max_area_found = 0

    for result in results:
        boxes = result.boxes.cpu().numpy()
        for box_data in boxes: # Đổi tên biến để tránh nhầm lẫn với 'box' của người gần nhất
            x1_curr, y1_curr, x2_curr, y2_curr = map(int, box_data.xyxy[0])
            conf_curr = box_data.conf[0]

            current_box_width = x2_curr - x1_curr
            current_box_height = y2_curr - y1_curr
            current_area = current_box_width * current_box_height

            if current_area > 0 and current_area > max_area_found:
                max_area_found = current_area
                largest_area_person_info = {
                    'x1': x1_curr, 'y1': y1_curr, 'x2': x2_curr, 'y2': y2_curr,
                    'conf': conf_curr,
                    'width': current_box_width,
                    'height': current_box_height,
                    'area': current_area
                }

    # Nếu tìm thấy một người, thì vẽ thông tin cho người có diện tích lớn nhất
    if largest_area_person_info is not None:
        # Lấy thông tin của người gần nhất
        x1 = largest_area_person_info['x1']
        y1 = largest_area_person_info['y1']
        x2 = largest_area_person_info['x2']
        y2 = largest_area_person_info['y2']
        conf = largest_area_person_info['conf']
        box_width_val = largest_area_person_info['width']
        box_height_val = largest_area_person_info['height']
        bounding_box_area_val = largest_area_person_info['area']

        # Chuẩn bị chuỗi hiển thị cho width, height, area với đơn vị pixel
        width_display_str = f"W: {box_width_val}px"
        height_display_str = f"H: {box_height_val}px"
        area_display_str = f"Area: {bounding_box_area_val}px²"

        distance_str = "Dist: N/A"
        if bounding_box_area_val > 0:
            distance = K_DISTANCE_FACTOR / bounding_box_area_val
            distance_str = f"Dist: {distance:.2f} m"

        # --- Vẽ bounding box ---
        cv2.rectangle(frame_bgr, (x1, y1), (x2, y2), (0, 0, 255), 3) # Màu đỏ và dày hơn cho người gần nhất

        # --- Hiển thị thông tin ---
        # 1. Label (Person) và Confidence
        label_info = f"Closest Person {conf:.2f}"
        y_pos_label = y1 - 35 if y1 > 35 else y1 + 15
        cv2.putText(frame_bgr, label_info, (x1, y_pos_label),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2) # Màu đỏ

        # 2. Distance
        y_pos_distance = y_pos_label + 20
        if y1 - 15 > 0 and y1 - 15 < y_pos_label:
             y_pos_distance = y1 - 15
        elif y_pos_distance < 20 :
             y_pos_distance = y_pos_label + 20
        cv2.putText(frame_bgr, distance_str, (x1, y_pos_distance),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 100, 0), 2)

        # --- Hiển thị Width, Height, Area bên dưới Distance ---
        current_y_for_details = y_pos_distance

        current_y_for_details += LINE_SPACING_DETAILS
        cv2.putText(frame_bgr, width_display_str, (x1, current_y_for_details),
                    FONT_DETAILS, FONT_SCALE_DETAILS, COLOR_DETAILS, THICKNESS_DETAILS)

        current_y_for_details += LINE_SPACING_DETAILS
        cv2.putText(frame_bgr, height_display_str, (x1, current_y_for_details),
                    FONT_DETAILS, FONT_SCALE_DETAILS, COLOR_DETAILS, THICKNESS_DETAILS)

        current_y_for_details += LINE_SPACING_DETAILS
        cv2.putText(frame_bgr, area_display_str, (x1, current_y_for_details),
                    FONT_DETAILS, FONT_SCALE_DETAILS, COLOR_DETAILS, THICKNESS_DETAILS)

    cv2.imshow("YOLO Detection - Closest Person", frame_bgr)

    if cv2.waitKey(1) & 0xFF == 27:  # Nhấn phím ESC để thoát
        break

# --- Giải phóng tài nguyên ---
cap.release()
cv2.destroyAllWindows()
print("Đã đóng webcam và giải phóng tài nguyên.")

Webcam đã được mở. Nhấn 'ESC' để thoát.
Đã đóng webcam và giải phóng tài nguyên.
